In [1]:
%load_ext autoreload
%autoreload 2
from notebook2script import *

# Training structure

In [2]:
#export
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

def training(train_loader, test_loader, model, optim_algo, learn_rate, device=torch.device('cpu'), epochs=20):
    nll_train = []
    nll_test = []
    optimizer = optim_algo(model.parameters(), learn_rate)
    test_loss = evaluate(test_loader, model, device)
    nll_test.append(test_loss.item())
    for epoch in range(epochs):
        loss_list = train(train_loader, model, optimizer, learn_rate, device)
        nll_train += loss_list
        test_loss = evaluate(test_loader, model, device)
        nll_test.append(test_loss.item())
        print(f'Epoch {epoch} loss train: {loss_list[-1]}, test: {nll_test[-1]}')
    return nll_train, nll_test
        
def train(train_loader, model, optimizer, learn_rate, device):
    model.train()
    loss_list = []
    for batch in train_loader:
        batch = batch.to(device)
        logits = model(batch)
        loss = model.loss_function(logits, batch) 
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        loss_list.append(loss.item())
    return loss_list

def evaluate(test_loader, model, device):
    model.eval()
    with torch.no_grad():
        loss = 0
        for batch in test_loader:
            batch = batch.to(device)
            logits = model(batch)
            loss += model.loss_function(logits, batch) * batch.shape[0]
        total_loss = loss / len(test_loader.dataset)
    return total_loss
        
def get_loaders(train_data, test_data, bs):
    train_loader = data.DataLoader(torch.as_tensor(train_data), batch_size=bs, shuffle=True)
    test_loader = data.DataLoader(torch.as_tensor(test_data), batch_size=bs, shuffle=False)
    return train_loader, test_loader

### Export

In [3]:
nb2script('Utils')

Converted Utils.ipynb to /home/magda/Github/deepul/deepul/exp_utils.py
